#### **Projekt1**

Pre celkovo najlepšie natrénovanú CNN, zobrazte vybrané filtre v 1. a 2. konvolučnej vrstve, zobrazte výslednú mapu príznakov pre vybraný vstupný obraz. 


In [ ]:
import pickle
import numpy as np
import torch
from pretty_confusion_matrix import pp_matrix_from_data
from models_architecture import *
from matplotlib import pyplot as plt
from torchvision import utils

Best network

In [ ]:
#modify accordingly
model = resnet18_architecture()
name = model.__class__.__name__
run='1'

path = 'trained_models'


model.load_state_dict(torch.load(f"{path}/{name}/{run}/model.pt"))

In [ ]:

def visTensor(tensor, ch=0, allkernels=False, nrow=8, padding=1,title="Convolution layer filters"):
    n, c, w, h = tensor.shape

    if allkernels:
        tensor = tensor.view(n * c, -1, w, h)
    elif c != 3:
        tensor = tensor[:, ch, :, :].unsqueeze(dim=1)

    rows = np.min((tensor.shape[0] // nrow + 1, 64))
    grid = utils.make_grid(tensor, nrow=nrow, normalize=True, padding=padding)
    plt.figure(figsize=(nrow, rows))
    plt.imshow(grid.numpy().transpose((1, 2, 0)))
    plt.title(title)
    # plt.show()

Filters of convolutional layers 1 and 2

In [ ]:
print(model)

print("Displaying layer 1")
filter_1 = model.conv1.weight.data.clone().cpu()
%matplotlib qt
visTensor(filter_1, ch=0, allkernels=False, title="Convolution layer 1 filters")

print("Displaying layer 2")
filter_2 = model.layer1[0].conv1.weight.data.clone().cpu()
visTensor(filter_2, ch=0, allkernels=False, title="Convolution layer 2 filters")


#filter = cnn2.conv1[0].weight.data.clone().cpu()
#visTensor(filter, ch=0, allkernels=False)